<h1><center>Story of what happens in American's life</center></h1>

It is a well-known fact that a complete rotation of the planet Earth takes 24 hours causing the formation of day and night. The fascinating part is to analyze how the people in the world decide to spend the available precious 24 hours. The demographics plays a very vital role on analysing and understanding the life-style of people mainly due to the influence of the time-zone, culture, population, socio-economy factors, current trend, etc. In this article, we focus on visually studying the behaviour of person living in the United States for the most obvious reasons that the country comprises of diverse, multi-cultural and a democratic-republic population.

To go another step ahead, this article shows how an American spends his/her day which is classified into a number of activities like Personal Care, Travelling, Socialising-Leisure time, Religion, Work, Sports- Fitness and so on.


In [1]:
#nbi:hide_in
import pandas as pd
import numpy as np
import re
import bqplot as bq
import ipywidgets
from ipywidgets import Layout

In [2]:
#nbi:hide_in
#nbi:hide_out
ats_sum = pd.read_csv('https://raw.githubusercontent.com/sarvaniputta/nbinteract_tutorial/master/atussum_2017.csv')
cps = pd.read_csv('https://raw.githubusercontent.com/sarvaniputta/nbinteract_tutorial/master/atuscps_2017.csv')
cps_ = cps.groupby('TUCASEID').GESTFIPS.first().to_frame()

merged = ats_sum.merge(cps_, left_on = 'TUCASEID', right_index = True)

# Code for extrating relevant activity columns
def activity_columns(data, activity_code):
    col_prefix = "t" + activity_code
    return [column for column in data.columns if col_prefix in column]

## Statewise data for major activities

work_cols = activity_columns(merged, '0501')
travel_cols = activity_columns(merged, '1805')
sleep_cols = activity_columns(merged, '0101')
religion_cols = activity_columns(merged, '1401')
leisure_cols = activity_columns(merged, '1203')
sports_cols = activity_columns(merged, '1301')
housework_cols = activity_columns(merged, '0201')

work_statewise = merged.loc[:, work_cols].groupby(merged.GESTFIPS).mean()
travel_statewise = merged.loc[:, travel_cols].groupby(merged.GESTFIPS).mean()
sleep_statewise = merged.loc[:, sleep_cols].groupby(merged.GESTFIPS).mean()
religion_statewise = merged.loc[:, religion_cols].groupby(merged.GESTFIPS).mean()
leisure_statewise = merged.loc[:, leisure_cols].groupby(merged.GESTFIPS).mean()
sports_statewise = merged.loc[:, sports_cols].groupby(merged.GESTFIPS).mean()
housework_statewise = merged.loc[:, housework_cols].groupby(merged.GESTFIPS).mean()

activity_cols_dict = {0: work_cols, 1: travel_cols, 2: sleep_cols, 3: religion_cols, 4: leisure_cols,
                      5: sports_cols, 6: housework_cols}

In [3]:
#nbi:hide_in
activity_list = ['Average Working Time', 'Average Travel Time', 'Average Sleeping Time', 'Average Religious Time',
                'Average Leisure Time', 'Average Sports Time', 'Average Housework Time']

act_dd = ipywidgets.Dropdown(options = activity_list, description = 'Select activity', 
                             style={'description_width': 'initial'})


######################### Map ############################
cscale = bq.ColorScale(scheme = 'RdPu')        
                   # reverse the colorscale or Hawaii is not visible

map_tt = bq.Tooltip(labels = ['State', 'Time (minutes)'], fields = ['name', 'color'])
sc_geo = bq.AlbersUSA(scale_factor = 1650)
states_map = bq.Map(color = work_statewise.sum(axis=1).round(1).to_dict(), 
                     map_data=bq.topo_load('map_data/USStatesMap.json'),
                    scales = {'projection':sc_geo, 'color':cscale},tooltip = map_tt,
                    interactions = {'click': 'select', 'hover':'tooltip', },
                    anchor_style = {'fill':'red'}, 
                    selected_style = {'opacity': 1.0},
                    unselected_style = {'opacity': 1.0})

cax = bq.ColorAxis(scale=cscale, orientation='horizontal', side='bottom', label='Time(minutes)')
fig_map = bq.Figure(marks=[states_map],axes=[cax], title = 'Average Working Time',
                   layout=Layout(min_width='52em', min_height='50em'),
                   background_style = {'fill': 'white'},
                    title_style = {'font-family': 'Montserrat', 'font-size': '30px',
                                  'text-transform': 'uppercase', 'font-weight': '700', 'color': '#E84A27'},)
fig_map.fig_margin = {'bottom': 10, 'top': 0, 'left': 0, 'right': 0}

###########################################################

######################### Bar #############################
time_spent = merged.loc[:, ['TUCASEID'] + work_cols].set_index('TUCASEID').sum(axis=1)

def normalized_hist(data, bins=10):
    counts, bins = np.histogram(data, bins=bins)
    counts = counts*100/counts.sum()
    return bins, counts  

sc_x2 = bq.LinearScale()
sc_y2 = bq.LinearScale()
ax_x2 = bq.Axis(label = 'Time spent for work(minutes)', scale=sc_x2, orientation='horizontal')
ax_y2 = bq.Axis(label='Frequency%', scale=sc_y2,orientation='vertical', grid_color='gray', grid_lines='dashed')
x, y = normalized_hist(time_spent, bins=24)
bars_hist = bq.Bars(x = x[:-1], y=y, align='right', scales={'x': sc_x2, 'y': sc_y2}, colors=['#54006d'])
#           ,width=bins[1]-bins[0]  , edgecolor='black')
fig_bar1 = bq.Figure(marks = [bars_hist],axes=[ax_x2, ax_y2], background_style = {'fill': 'white'})
#fig_bar1.fig_margin = {'bottom':50, 'top':50, 'left':20, 'right':10}

###########################################################

##################### Age Line Plot #######################
age_wise_act_sum = merged.groupby('TEAGE').sum()[work_cols].sum(axis=1)
age_wise_count = merged.groupby('TEAGE').count()['TUCASEID']
y_data_line = (age_wise_act_sum / age_wise_count).values
x_data_line = sorted(ats_sum['TEAGE'].unique())

x_sc_line = bq.OrdinalScale()
y_sc_line = bq.LinearScale()

x_ax_line = bq.Axis(scale = x_sc_line, num_ticks=20, label='Age')
y_ax_line = bq.Axis(scale = y_sc_line, 
                   orientation = 'vertical', label='Time spent in minutes',
                    grid_color='gray', grid_lines='dashed')

lines = bq.Lines(x = x_data_line, y = y_data_line, scales = {'x': x_sc_line, 'y': y_sc_line},
                colors=['#54006d'])

fig_line = bq.Figure(marks = [lines], axes = [x_ax_line, y_ax_line], 
                    title='Avg. time spent in activity per age', 
                     title_style = {'font-family': 'Montserrat'}, background_style = {'fill': 'white'}) 

###########################################################

##################### Interactivity #######################

def on_select_map(change):
    if not change['new']:
        selected_fips = merged.GESTFIPS.unique()
    else:
        selected_fips = change['new']
        states_map.selected = [selected_fips[-1]]
    activity = act_dd.index
    columns = activity_cols_dict[activity]
    x, y = normalized_hist(merged
                            .loc[merged.GESTFIPS.isin([selected_fips[-1]]), ['TUCASEID']+columns]
                            .set_index('TUCASEID')
                            .sum(axis=1), bins=24)
    bars_hist.x = x
    bars_hist.y = y
    # For updating line plot
    #merged_subset = merged[merged['GESTFIPS'] == selected_fips[-1]]
    #age_wise_act_sum = merged_subset.groupby('TEAGE').sum()[work_cols].sum(axis=1)
    #age_wise_count = merged_subset.groupby('TEAGE').count()['TUCASEID']
    #lines.y = (age_wise_act_sum / age_wise_count).values
    #lines.x = sorted(age_wise_act_sum.index)
    
    #print(selected_fips) Fix needed for selectex FIPS [50, 36]
# Observe above changes 
states_map.observe(on_select_map, 'selected')

def on_activity_change(change):
    states_map.selected=[]
    activity = act_dd.index
    columns = activity_cols_dict[activity]
    filtered = merged.loc[:, columns].groupby(merged.GESTFIPS).mean().sum(axis=1).round(1).to_dict()
    states_map.color = filtered
    fig_map.title = act_dd.value
    # For updating line plot
    age_wise_act_sum = merged.groupby('TEAGE').sum()[columns].sum(axis=1)
    age_wise_count = merged.groupby('TEAGE').count()['TUCASEID']
    lines.y = (age_wise_act_sum / age_wise_count).values
    lines.x = sorted(ats_sum['TEAGE'].unique())
    
# Observe above changes 
act_dd.observe(on_activity_change, 'value')

###########################################################

ipywidgets.VBox([act_dd, ipywidgets.HBox([fig_map, ipywidgets.VBox([fig_bar1, fig_line])])])